In [2]:
# 셀레니움
from selenium import webdriver 
# 셀레니움 By 클래스 가져오기
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
import time
import random
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
# NoSuchElementException import 추가
from selenium.common.exceptions import NoSuchElementException  
import pandas as pd
from selenium.common.exceptions import StaleElementReferenceException
import logging
from selenium.common.exceptions import TimeoutException, WebDriverException
from datetime import datetime
import re

# 창 열기 1번 실행

- 키워드 입력 부분에 검색어 바꿔주기

In [3]:
# 크롬 웹 드라이버 초기화 및 카카오 맵 페이지로 이동
driver = webdriver.Chrome() 
url = 'https://m.map.kakao.com/'
driver.get(url)

# 검색창 element와 검색창 id 가져오기
search_box = driver.find_element(By.ID,'innerQuery')

# 키워드 입력
# 키워드 바꿔주기
search_box.send_keys("성북구 빵")

# Enter 키 누르기
search_box.send_keys(Keys.RETURN)

# 페이지가 로드될 때까지 잠시 대기
time.sleep(3)

# 베너 닫기
driver.find_element(By.XPATH, f'//*[@id="daumWrap"]/div[1]/div/a[2]/span').click()

# 더보기 버튼을 누를 수 있는지 여부 확인하는 함수
def can_click_load_more_button():
    try:
        load_more_button = driver.find_element(By.XPATH, '//*[@id="daumContent"]/div[4]/div[2]/div[3]/a')  # 더보기 버튼의 XPATH
        return load_more_button.is_enabled() and load_more_button.is_displayed()
    except:
        return False

# 더보기 버튼 클릭 함수
def click_load_more_button():
    load_more_button = driver.find_element(By.XPATH, '//*[@id="daumContent"]/div[4]/div[2]/div[3]/a')  # 더보기 버튼의 XPATH
    load_more_button.click()
    time.sleep(3)  # 버튼을 클릭한 후에 충분한 시간 대기 (페이지가 로딩될 때까지)

# 더보기 버튼을 누를 수 없을 때까지 반복해서 클릭
while can_click_load_more_button():
    click_load_more_button()
    
# 페이지에 표시된 정보의 갯수 확인
all_results = driver.find_elements(By.XPATH, '//*[@id="placeList"]/li')
print("페이지에 표시된 정보의 갯수:", len(all_results))

driver.execute_script("window.scrollTo(0, -document.body.scrollHeight);")

페이지에 표시된 정보의 갯수: 76


# 정보 크롤링 1번만 실행

In [4]:
# 크롤링
# 가게 이름
def store_def(index):
    xpath = f'//*[@id="placeList"]/li[{index}]/a[1]/span/span[1]/strong'
    try:
        return driver.find_element(By.XPATH, xpath).text
    # 베너가 있는 경우 다른 XPATH로 시도
    except NoSuchElementException:
        xpath = f'//*[@id="placeList"]/li[{index}]/a/span/span[1]/strong'   
        
        return driver.find_element(By.XPATH, xpath).text
    
# 리뷰 점수  
def rv_score_def(index):
    xpath = f'//*[@id="placeList"]/li[{index}]/a[1]/span/span[2]/span[1]/em'
    try:
        return driver.find_element(By.XPATH, xpath).text
    except NoSuchElementException:
        xpath = f'//*[@id="placeList"]/li[{index}]/a/span/span[2]/span[1]/em'
     
        try:
            return driver.find_element(By.XPATH, xpath).text
        # 예외 처리: 요소를 찾지 못할 경우 None 반환
        except NoSuchElementException:
            return None 


# 리뷰 갯수 
def rv_count_def(index):
    xpath = f'//*[@id="placeList"]/li[{index}]/a[1]/span/span[2]/span[2]'
    try:
        return driver.find_element(By.XPATH, xpath).text
    except NoSuchElementException:
        xpath = f'//*[@id="placeList"]/li[{index}]/a/span/span[2]/span[2]'
        try:
            return driver.find_element(By.XPATH, xpath).text
        except NoSuchElementException:
            return None 

# 주소 
def add_def(index):
    xpath = f'//*[@id="placeList"]/li[{index}]/a[1]/span/span[3]'
    try:
        return driver.find_element(By.XPATH, xpath).text
    except NoSuchElementException:
        xpath = f'//*[@id="placeList"]/li[{index}]/a/span/span[3]'
        return driver.find_element(By.XPATH, xpath).text

store = []
review_score = []
review_count = []
address = []

for i in range(1, len(all_results) + 1): 
    if i == 3:
        continue
    g = 0
    max_attempts = 3  # 최대 시도 횟수를 설정하여 무한 루프 방지
    attempts = 0
    time.sleep(3)
    try:
        store_name = store_def(i)
        store.append(store_name)
        
        review_sc = rv_score_def(i)
        review_score.append(review_sc)
        
        review_ct = rv_count_def(i)
        review_count.append(review_ct)
        
        add = add_def(i)
        address.append(add)
        driver.execute_script("window.scrollBy(0, 200);")
        g += 1
        attempts = 0
    # 이 부분은 모든 요소에 대한 크롤링 시도가 실패했을 때를 처리
    except (WebDriverException or NoSuchElementException or TimeoutException):  
            attempts += 1

    if attempts >= max_attempts:
        print("최대 시도 횟수에 도달하였거나 더 이상 크롤링할 요소가 없습니다.")
        break

driver.execute_script("window.scrollTo(0, -document.body.scrollHeight);")

In [5]:
print(len(store))
print(len(review_score))
print(len(review_count))
print(len(address))

75
75
75
75


### 리뷰 갯수 정수형으로 바꾸기

In [8]:
# 숫자로 변환하는 함수
def extract_number(item):
    if item is None:
        return None
    match = re.search(r'\d+', item)
    return int(match.group()) if match else None

# 각 항목을 숫자로 변환하여 새로운 리스트 생성
review_count = [extract_number(item) for item in review_count]

# 정보데이터 DF에 저장하기

- 필요하다면 DF 이름 바꾸주기

In [9]:
# DF 생성
sung_info = pd.DataFrame()

# '구'까지 슬라이싱해서 'addr' 칼럼에 추가하는 함수
def slice_to_gu(address):
    gu_index = address.find('구') + 1
    if gu_index:
        return address[:gu_index]
    else:
        return address

sung_info['Platform'] = ['Kakao' for _ in range(len(all_results)-1)]
sung_info['Number'] = [i for i in range(1, len(all_results))]
sung_info['Store'] = store
sung_info['Address'] = address
# Address 칼럼에서 '구'까지 스플릿
sung_info['Addr'] = sung_info['Address'].apply(lambda x: slice_to_gu(x))
sung_info['Review_score'] = review_score
sung_info['Review_counts'] = review_count

sung_info

,Platform,Number,Store,Address,Addr,Review_score,Review_counts
0,Kakao,1,나폴레옹과자점 성북본점,서울 성북구 성북로 7 (성북동1가),서울 성북구,3.0,162.0
1,Kakao,2,밀곳간,서울 성북구 성북로18길 19 1층 (성북동),서울 성북구,4.1,136.0
2,Kakao,3,그레도제빵사,서울 성북구 화랑로 304 (석관동),서울 성북구,3.9,133.0
3,Kakao,4,우주제빵소 (휴업중),"서울 성북구 성북로28길 13 지하1층,1,2층 (성북동)",서울 성북구,3.3,48.0
4,Kakao,5,달콤한위로,"서울 성북구 창경궁로43길 24 1,2층 (성북동1가)",서울 성북구,3.7,51.0
...,...,...,...,...,...,...,...
70,Kakao,71,더베이크,서울 성북구 안암로 145 (안암동5가),서울 성북구,5.0,2.0
71,Kakao,72,파리바게뜨 동덕여대점,서울 성북구 화랑로11길 7 (하월곡동),서울 성북구,3.6,5.0
72,Kakao,73,파리바게뜨 종암점,서울 성북구 종암로 96 (종암동),서울 성북구,3.0,7.0
73,Kakao,74,온세케이크,서울 성북구 한천로 660-16 장위지웰에스테이트 101동 상가 1층 119호 (장위동),서울 성북구,None,NaN


# 리뷰 크롤링 함수

In [10]:
# 랜덤한 대기 시간
wait = random.uniform(5,10)

# 로깅 설정
logging.basicConfig(level=logging.INFO, format='%(asctime)s - %(levelname)s - %(message)s')

# 가게이름 클릭 함수
def click_def(index):
    xpath = f'//*[@id="placeList"]/li[{index}]/a[1]/span[2]/span[1]/strong'
    try:
        driver.find_element(By.XPATH, xpath).click()
    # 베너가 있는 경우 다른 XPATH로 시도
    except NoSuchElementException:
        logging.info(f"No such element: {xpath}")
        xpath = f'//*[@id="placeList"]/li[{index}]/a/span[2]/span[1]/strong' 
        try:
            driver.find_element(By.XPATH, xpath).click()
        except NoSuchElementException:
            logging.info(f"No such element: {xpath}")
            return None

# 스크롤하여 요소가 보이도록 만드는 함수
def scroll_to_element(element):
    driver.execute_script("arguments[0].scrollIntoView(true);", element)

# 후기 더보기 누르기, 후기 접기는 정지
def class_click():
    prev_class = 'link_more'
    while True:
        element = driver.find_element(By.XPATH, f'//*[@id="mArticle"]/div[*]/div/div[2]/a')
        element_class = element.get_attribute("class")
        # 이전 클래스와 현재 클래스가 다른지 확인
        if prev_class != element_class:
            print("Class changed. Stopping click.")
            break
        # 클릭하고 잠시 대기
        element.click()
        time.sleep(wait)
        # 현재 클래스를 이전 클래스에 저장
        prev_class = element_class

# find_total 수정       
def find_data():
    g = 1  # 루프 밖에서 변수 초기화
    max_attempts = 3  # 최대 시도 횟수를 설정하여 무한 루프 방지
    attempts = 0
    while attempts < max_attempts:
        try:
            xpath_id = f'//*[@id="mArticle"]/div[*]/div/div[2]/ul/li[{g}]/div[1]/div[1]/div[1]/a/span'
            xpath_date = f'//*[@id="mArticle"]/div[*]/div/div[2]/ul/li[{g}]/div[1]/div[1]/div[2]/span[6]'
            xpath_rv = f'//*[@id="mArticle"]/div[*]/div/div[2]/ul/li[{g}]/div[1]/div[*]/p/span'

            try:
                id_element = WebDriverWait(driver, wait).until(EC.visibility_of_element_located((By.XPATH, xpath_id)))
                temp_ID = id_element.text.strip()
                if temp_ID:
                    ID.append(id_element.text)
                else:
                    ID.append('정보 없음')
            except (WebDriverException or NoSuchElementException or TimeoutException): 
                ID.append('정보 없음')
            time.sleep(wait)

            try:
                date_element = WebDriverWait(driver, wait).until(EC.visibility_of_element_located((By.XPATH, xpath_date)))
                Date.append(date_element.text)
            except (WebDriverException or NoSuchElementException or TimeoutException): 
                break

            time.sleep(wait)

            try:
                rv_element = WebDriverWait(driver, wait).until(EC.visibility_of_element_located((By.XPATH, xpath_rv)))
                temp_review_text = rv_element.text.strip()
                if temp_review_text:
                    Review_text.append(temp_review_text)
                else:
                    Review_text.append('정보 없음')
            except (WebDriverException or NoSuchElementException or TimeoutException):  
                Review_text.append('정보 없음')
            
            now = datetime.now().strftime('%Y-%m-%d - %H:%M:%S')
            time_list.append(now)   

            driver.execute_script("window.scrollBy(0, 200);")
            g += 1
            attempts = 0
        # 이 부분은 모든 요소에 대한 크롤링 시도가 실패했을 때를 처리
        except (WebDriverException or NoSuchElementException or TimeoutException):  
            attempts += 1

        if attempts >= max_attempts:
            print("최대 시도 횟수에 도달하였거나 더 이상 크롤링할 요소가 없습니다.")
            break

# 리뷰 크롤링 시작

In [14]:
ID = []
Date = []
Review_text = []
store = []
time_list = []

for index in range(1, len(all_results) + 1):
    wait_time = random.uniform(1, 5)

    # 가게 이름 클릭
    xpath = f'//*[@id="placeList"]/li[{index}]/a[1]/span[2]/span[1]/strong'
   
    try:
        driver.find_element(By.XPATH, xpath).click()
        time.sleep(wait_time)
        try:
            # 가게 이름 크롤링
            cw = driver.find_element(By.XPATH, '//*[@id="mArticle"]/div[1]/div/div[1]/h2').text
            # 후기 갯수 크롤링
            qa = driver.find_element(By.XPATH, '//*[@id="mArticle"]/div[1]/div/div[1]/div[1]/a[1]/span[2]/span[1]').text
            temp_store = [cw for _ in range(int(qa))]
            for i in temp_store:
                store.append(i)
            time.sleep(wait_time)
        except NoSuchElementException:
            store.append(driver.find_element(By.XPATH, '//*[@id="mArticle"]/div[1]/div/div[1]/h2').text)
            ID.append('정보 없음')
            Date.append('정보 없음')
            Review_text.append('정보 없음')
            driver.back()
            continue
    except NoSuchElementException:
        continue
    
    time.sleep(wait_time)
    
    # 스크롤 내릴 위치 찾기
    try:
        more_reviews_link = driver.find_element(By.CLASS_NAME, 'rate_intro')
    except NoSuchElementException:
        driver.back()
        continue
    time.sleep(wait_time)
    
    # 후기 더보기 링크가 보이도록 스크롤
    scroll_to_element(more_reviews_link)
    
    # 첫번째 리뷰누르기 - 전체 리뷰 페이지로 넘어가기
    try:
        driver.find_element(By.XPATH, f'//*[@id="mArticle"]/div[*]/div/ul[*]/li[1]/div[1]/div[2]/p').click()
    except NoSuchElementException:
        driver.back()
        continue
    time.sleep(wait_time)
    
    # 첫 번째 더보기 버튼 클릭
    try:
        class_click()
    except NoSuchElementException:
        print("Can't Click!")
        pass
    # 스크롤 올리기
    driver.execute_script("window.scrollTo(0, -document.body.scrollHeight);")
    # 크롤링 함수
    find_data()
    ID.pop()
    
    # 검색 결과 페이지로 돌아가기
    driver.back()
    driver.back()
    
    # 페이지가 로드될 때까지 잠시 대기
    time.sleep(wait_time)
    # 더보기 버튼
    while can_click_load_more_button():
        click_load_more_button()

Class changed. Stopping click.
Class changed. Stopping click.
Class changed. Stopping click.
Class changed. Stopping click.
Class changed. Stopping click.
Class changed. Stopping click.
Class changed. Stopping click.
Class changed. Stopping click.
Class changed. Stopping click.
Class changed. Stopping click.
Class changed. Stopping click.
Can't Click!
Class changed. Stopping click.
Class changed. Stopping click.
Class changed. Stopping click.
Can't Click!
Class changed. Stopping click.
Class changed. Stopping click.
Can't Click!
Class changed. Stopping click.
Class changed. Stopping click.
Class changed. Stopping click.
Class changed. Stopping click.
Can't Click!
Class changed. Stopping click.
Can't Click!
Class changed. Stopping click.
Class changed. Stopping click.
Class changed. Stopping click.
Can't Click!
Can't Click!
Can't Click!
Class changed. Stopping click.
Can't Click!
Class changed. Stopping click.
Can't Click!
Can't Click!
Can't Click!
Can't Click!
Can't Click!
Class change

In [21]:
print(len(store))
print(len(ID))
print(len(Date))
print(len(Review_text))
print(len(time_list))

1459
1459
1459
1459
1459


# 리뷰부분 데이터 프레임에 저장

In [22]:
eu = pd.DataFrame()
eu['Store'] = store
eu['ID'] = ID
eu['Date'] = Date
eu['Review_text'] = Review_text
eu['Time'] = time_list

eu

,Store,ID,Date,Review_text,Time
0,나폴레옹과자점 성북본점,이쁜애,2024.03.10.,난 맛이없어 왜?ㅠㅜㅠㅠㅠ,2024-04-04 - 23:20:17
1,나폴레옹과자점 성북본점,abc,2024.03.06.,비싼값하는 빵집\n맛있다,2024-04-04 - 23:20:33
2,나폴레옹과자점 성북본점,gestalt,2024.03.03.,오전 11시경에는 빵이 별로 없다.\n\n내가 고른 빵들은 모두 달았다\n\n유럽왕...,2024-04-04 - 23:20:49
3,나폴레옹과자점 성북본점,San-A,2024.03.01.,정보 없음,2024-04-04 - 23:21:12
4,나폴레옹과자점 성북본점,엥프피,2024.02.21.,파리바게트 상위호환,2024-04-04 - 23:21:28
...,...,...,...,...,...
1454,와플대학 길음역캠퍼스,김서현,2021.07.02.,다른 지점보다 크림을 더 많이 주시는 것 같아요☺☺아주머니도 친절하시구~ 오래오래 ...,2024-04-05 - 07:05:08
1455,와플대학 길음역캠퍼스,김덕팔,2021.01.17.,넘 맛있고 사장님이 친절하세요! 크림 낭낭 와플 바삭,2024-04-05 - 07:05:24
1456,와플대학 길음역캠퍼스,van,2020.10.05.,크림 진짜 너무 많이줌 너무 좋아,2024-04-05 - 07:05:40
1457,와플대학 길음역캠퍼스,챠이,2020.09.04.,집이 정반대라 자주는 못 가고 약속 있을 때만 들리는데 너무너무 맛있어요! 길음역 ...,2024-04-05 - 07:05:56


In [30]:
# eu.to_csv('eu.csv')
# 저장 된거

# 정보 DF랑 리뷰 DF merge

- 정보 DF를 앞, 리뷰 DF를 뒤에 써주세요

In [53]:
# 정보랑 리뷰 merge
eunpyeong = pd.merge(sung_info, wer, on = 'Store')
eunpyeong

,Platform,Number,Store,Address,Addr,Review_score,Review_counts,ID,Date,Review_text,Time
0,Kakao,1,나폴레옹과자점 성북본점,서울 성북구 성북로 7 (성북동1가),서울 성북구,3.0,162.0,이쁜애,2024.03.10.,난 맛이없어 왜?ㅠㅜㅠㅠㅠ,2024-04-04 - 23:20:17
1,Kakao,1,나폴레옹과자점 성북본점,서울 성북구 성북로 7 (성북동1가),서울 성북구,3.0,162.0,abc,2024.03.06.,비싼값하는 빵집\n맛있다,2024-04-04 - 23:20:33
2,Kakao,1,나폴레옹과자점 성북본점,서울 성북구 성북로 7 (성북동1가),서울 성북구,3.0,162.0,gestalt,2024.03.03.,오전 11시경에는 빵이 별로 없다.\n\n내가 고른 빵들은 모두 달았다\n\n유럽왕...,2024-04-04 - 23:20:49
3,Kakao,1,나폴레옹과자점 성북본점,서울 성북구 성북로 7 (성북동1가),서울 성북구,3.0,162.0,San-A,2024.03.01.,정보 없음,2024-04-04 - 23:21:12
4,Kakao,1,나폴레옹과자점 성북본점,서울 성북구 성북로 7 (성북동1가),서울 성북구,3.0,162.0,엥프피,2024.02.21.,파리바게트 상위호환,2024-04-04 - 23:21:28
...,...,...,...,...,...,...,...,...,...,...,...
1470,Kakao,75,와플대학 길음역캠퍼스,서울 성북구 동소문로 248 삼부아파트상가 105동 1층 9호 (길음동),서울 성북구,4.2,10.0,김서현,2021.07.02.,다른 지점보다 크림을 더 많이 주시는 것 같아요☺☺아주머니도 친절하시구~ 오래오래 ...,2024-04-05 - 07:05:08
1471,Kakao,75,와플대학 길음역캠퍼스,서울 성북구 동소문로 248 삼부아파트상가 105동 1층 9호 (길음동),서울 성북구,4.2,10.0,김덕팔,2021.01.17.,넘 맛있고 사장님이 친절하세요! 크림 낭낭 와플 바삭,2024-04-05 - 07:05:24
1472,Kakao,75,와플대학 길음역캠퍼스,서울 성북구 동소문로 248 삼부아파트상가 105동 1층 9호 (길음동),서울 성북구,4.2,10.0,van,2020.10.05.,크림 진짜 너무 많이줌 너무 좋아,2024-04-05 - 07:05:40
1473,Kakao,75,와플대학 길음역캠퍼스,서울 성북구 동소문로 248 삼부아파트상가 105동 1층 9호 (길음동),서울 성북구,4.2,10.0,챠이,2020.09.04.,집이 정반대라 자주는 못 가고 약속 있을 때만 들리는데 너무너무 맛있어요! 길음역 ...,2024-04-05 - 07:05:56


# merge 시킨거 저장

- kakao_구이름 으로 저장

In [54]:
eunpyeong.to_csv('kakao_seongbuk.csv')


# 만약 중간에 멈추는 경우에만 할 것

- 카카오는 중간에 하나씩 안되는 가게가 있어요.

그럼 그 가게 클릭만 해서 들어가서 이거 눌러주세요. 

알아서 리뷰 크롤링하고 검색 페이지로 돌아갑니다.

오류난 가게 크롤링 후 위의 <리뷰 크롤링 시작>의 범위 다시 지정해주세요!

for index in range(1, len(all_results) + 1):<<< 이 부분의 시작범위 변경해주세요!


In [40]:
ID = []
Date = []
Review_text = []
store = []
time_list = []

wait_time = random.uniform(1, 5)

# 가게 이름 크롤링
cw = driver.find_element(By.XPATH, '//*[@id="mArticle"]/div[1]/div/div[1]/h2').text
# 후기 갯수 크롤링
qa = driver.find_element(By.XPATH, '//*[@id="mArticle"]/div[1]/div/div[1]/div[1]/a[1]/span[2]/span[1]').text
temp_store = [cw for _ in range(int(qa))]
for i in temp_store:
    store.append(i)
time.sleep(wait_time)



time.sleep(wait_time)

# 스크롤 내릴 위치 찾기
try:
    more_reviews_link = driver.find_element(By.CLASS_NAME, 'rate_intro')
except NoSuchElementException:
    driver.back()

time.sleep(wait_time)

# 후기 더보기 링크가 보이도록 스크롤
scroll_to_element(more_reviews_link)

# 첫번째 리뷰누르기 - 전체 리뷰 페이지로 넘어가기
try:
    driver.find_element(By.XPATH, f'//*[@id="mArticle"]/div[*]/div/ul[*]/li[1]/div[1]/div[2]/p').click()
except NoSuchElementException:
    driver.back()

time.sleep(wait_time)

# 첫 번째 더보기 버튼 클릭
try:
    class_click()
except NoSuchElementException:
    print("Can't Click!")
    pass
# 스크롤 올리기
driver.execute_script("window.scrollTo(0, -document.body.scrollHeight);")
# 크롤링 함수
find_data()
ID.pop()

# 검색 결과 페이지로 돌아가기
driver.back()
driver.back()

Class changed. Stopping click.


In [41]:
print(len(store))
print(len(ID))
print(len(Date))
print(len(Review_text))
print(len(time_list))

15
15
15
15
15
